In [1]:
%gui asyncio

In [2]:
import yaml
import asyncio
from ocs_academic_hub import HubClient

In [3]:
%env OCS_HUB_CONFIG=config-dv.ini

hub_client = HubClient()

env: OCS_HUB_CONFIG=config-dv.ini
> configuration file: config-dv.ini


In [4]:
with open("ucdavis-building-ceed-dv.yaml", "r") as stream:
    building_ceeds_dvid = yaml.safe_load(stream)
# list(building_ceeds_dvid.keys())
# with open("deschutes-dv.yaml") as stream:
#    deschutes_dvid = yaml.safe_load(stream)
deschutes_dvid = {f"Fermenter #{i}": {  }  for i in range(31, 37)}
for k in deschutes_dvid.keys():
    fv_id = k[-2:]
    for dv in ["ADF_Prediction", "Cooling_Prediction", "PCA"]:
        deschutes_dvid[k][dv] = f"HubDV_{dv}_fv{fv_id}"
# deschutes_dvid
# building_ceeds_dvid = deschutes_dvid
namespace_id = "UC__Davis"  # "UC__Davis"

In [5]:
from ipywidgets import ToggleButton, DatePicker, Select, interactive, Layout, Output
from IPython.display import display
import datetime

![](https://academichub.blob.core.windows.net/images/plain_blue_banner_v2.png)

#### To learn more about OSIsoft Academic Hub, click [here](https://academichub.blob.core.windows.net/images/PDF%20for%20CSV%20GUI%20-%20V%201.1.pdf)

## Steps to download a CSV 

1. Select an Asset 
2. Select a Data View
3. Pick Start and End Dates
4. Click **Download CSV** to download a result CSV with requested data

NOTE: to download another CSV, reload this web page

In [6]:
def wait_for_change(widget, value):
    future = asyncio.Future()

    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)

    widget.observe(getvalue, value)
    return future

In [10]:
def select_ceed(asset):
    ceedW.options = building_ceeds_dvid[asset].keys()


k = ToggleButton(
    value=False,
    description="Download CSV",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Description",
    icon="check",
)

init_start = datetime.date(2017, 3, 17)
init_end = datetime.date(2017, 9, 17)
start = DatePicker(description="Start Date", disabled=False, value=init_start)
end = DatePicker(description="End Date", disabled=False, value=init_end)


buildingW = Select(
    options=building_ceeds_dvid.keys(), layout=Layout(width="40%", height="120px")
)
init = buildingW.value
ceedW = Select(options=building_ceeds_dvid[init])


def print_ceed(dataview):
    pass
    # print(buildingW.value, dataview)


j = interactive(print_ceed, dataview=ceedW)
i = interactive(select_ceed, asset=buildingW)

out = Output()


def my_append_stdout(message, end="\n"):
    out.append_stdout(message + end)


async def f():
    out.append_stdout(
        "Click *Download CSV* button to get result CSV data with current parameters\n"
    )
    x = await wait_for_change(k, "value")
    out.append_stdout("working on getting data from OCS...\n")
    # status = load_extract_transfer(
    #     upload.value, api_key=api_key.value, timestamp_format=timestamp_format.value
    # )
    out.append_stdout(
        f"[ {buildingW.value} | {ceedW.value} | {start.value} | {end.value} | {building_ceeds_dvid[buildingW.value][ceedW.value]} ]\n"
    )
    with out:
        df = hub_client.dataview_interpolated_pd(namespace_id, building_ceeds_dvid[buildingW.value][ceedW.value], str(start.value), str(end.value), interval="00:30:00")
    out.append_stdout(f"df={df}\n")
    
    out.append_stdout(f"Download status OK\n")


asyncio.ensure_future(f())


display(i)
display(j)
display(start)
display(end)
display(k)
display(out)

interactive(children=(Select(description='asset', layout=Layout(height='120px', width='40%'), options=('Fermen…

interactive(children=(Select(description='dataview', options=('ADF_Prediction', 'Cooling_Prediction', 'PCA'), …

DatePicker(value=datetime.date(2017, 3, 17), description='Start Date')

DatePicker(value=datetime.date(2017, 9, 17), description='End Date')

ToggleButton(value=False, description='Download CSV', icon='check', tooltip='Description')

Output()